# Prompt templates

In previous recipes, a prompt was just a simple Python string. We already encountered a situation, where we need to use a variable in the prompt. For example, let's say we want to create a pun generator that creates a pun based on a general topic. Every time we prompt the model, only the topic part of the prompt will change. So what is an efficient, convenient way to handle this?

In [32]:
from langchain_dartmouth.llms import ChatDartmouth
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

## Building prompts with basic Python strings

As we have done before, we could create a simple string prompt and add the topic to it through string concatenation. First, we define the part of the prompt that does not change:

In [33]:
prompt = "You are a pun generator. Your task is to generate a pun based on the following topic: "

Then, we add the missing piece when we prompt the model:

In [34]:
llm = ChatDartmouth(model_name="llama-3-1-8b-instruct")
response = llm.invoke(prompt + "computer programming")

print(response.content)

I'm ready to compile a pun for you. Here's one:

Why do programmers prefer dark mode? Because light attracts bugs.

Hope that one didn't crash your day!


That works, but it is a little clunky. The main issue here is that we have to design the prompt in a way that puts all the variable parts at the end. For short prompts like this one, this might be acceptable. It greatly limits our design space, though, when we are dealing with longer instructions. What if want more than one variable part with a constant part in-between?


## Prompt Templates

Prompt templates (e.g., the [`PromptTemplate` class](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.prompt.PromptTemplate.html)) are components in the LangChain ecosystem that allow you to define your prompts more flexibly by using placeholders and then filling them with actual values when needed.

Let's create the same pun generator as above using a `PromptTemplate`:

In [35]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="You are a pun generator. Your task is to generate a pun based on the following topic: {topic}"
)

Notice the special substring `{topic}`! This is how we define a location and a name for a placeholder in the prompt!

<div class="alert alert-info" role="alert">

**Note:**
Prompt templates are similar to [Python's f-strings or format strings](https://docs.python.org/3/tutorial/inputoutput.html#formatted-string-literals), but offer some additional convenience when using them with other LangChain components, as we will see in some later recipes.
</div>

We can fill in the placeholder using the `PromptTemplate` component's `invoke` method to fully specify the prompt:

In [36]:
print(prompt.invoke("computer science"))

text='You are a pun generator. Your task is to generate a pun based on the following topic: computer science'


We can pass the now complete prompt directly to our LLM:

In [37]:
response = llm.invoke(prompt.invoke("computer science"))
print(response.content)

Why not "Byte-Sized Problem" for the programmer who's got a small issue to tackle?


So if we want to run this repeatedly for different topics, we only need to change the prompt template's argument:

In [38]:
topics = ["college", "soccer", "cooking"]

for topic in topics:
    response = llm.invoke(prompt.invoke(topic))
    print(response.content)
    print("-" * 10)

I've got a "degree" of puns for you. Here's one: 

"Why did the college student bring a ladder to class? He wanted to reach his full potential."

How's that? I'm "graduating" to the next pun...
----------
Here's a soccer pun for you:

"Why did the soccer player bring a pillow onto the field? He wanted to have a soft defense!"

Hope that one scored a goal with you!
----------
I'm egg-static to generate a pun for you. Here's one: 

"Why was the pizza in a bad mood? It was feeling a little crusty!"

Hope that one rises to the occasion!
----------


We could also extend this technique to multiple placeholders. Here is what the prompt template would look like in that case:

In [39]:
prompt = PromptTemplate(
    template="You are a pun generator. Your task is to generate a pun based on the following topic: {topic}. Your current mood is {mood}."
)

Now that we have more than placeholder, we cannot simply pass a single argument to the `invoke` method, though, because the prompt would not know which placeholder to map it to. Instead, we pass in a dictionary, using the placeholder names as keys and the desired text to fill-in as values:

In [40]:
placeholder_fillers = {"topic": "computer science", "mood": "exhilirated"}
print(prompt.invoke(placeholder_fillers))

text='You are a pun generator. Your task is to generate a pun based on the following topic: computer science. Your current mood is exhilirated.'


Now we can iterate through two lists t of topics and moods to generate pun for each pair:

In [41]:
moods = ["giggly", "dramatic", "whimsical"]

for topic, mood in zip(topics, moods):
    response = llm.invoke(prompt.invoke({"topic": topic, "mood": mood}))
    print(response.content)
    print("-" * 10)

*giggles* Okay, let's get this degree of puns going! 

Here's one: Why did the college student bring a ladder to class? Because they wanted to reach their full potential!
----------
*Sigh* Oh, the agony and the ecstasy of crafting a pun worthy of the beautiful game. Very well, I shall attempt to score a goal with my words. Here's a soccer pun, dripping with drama:

"Why did the soccer player bring a pillow onto the field? He wanted a soft defense, and to have a ball-anced attack, but ultimately, it was a match that left him feeling deflated..."

*Takes a deep breath* There, I've unleashed my pun upon the world. May it bring a smile to the face of the soccer enthusiast, or may it leave them feeling like they've just been tackled to the ground...
----------
What a "saucy" task you've given me. Here's a pun that's the "icing on the cake":

Why did the chef quit his job? He couldn't "whisk" his responsibilities away – he was "buttering" under the pressure!

I hope that one "stirs" up a smi

Since `PromptTemplate` objects are more than just strings, they have [a few methods and fields](https://api.python.langchain.com/en/latest/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate) that can be useful in the right circumstances. For example, you can learn the names of the required placeholders using the field `input_variables`:

In [43]:
prompt.input_variables

['mood', 'topic']

### Chat prompt templates

You can also create and use templates for chat prompts with a sequence of messages of different types:

In [52]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [("system", "You are a {animal}."), ("human", "Tell us about {topic}.")]
)

prompt.invoke({"animal": "dog", "topic": "your day"})

ChatPromptValue(messages=[SystemMessage(content='You are a dog.'), HumanMessage(content='Tell us about your day.')])

In [57]:
response = llm.invoke(prompt.invoke({"animal": "dog", "topic": "your day"}))
print(response.content)

My day was so much fun! I woke up early this morning and ran straight to my favorite spot on the couch. My humans always sit there in the mornings, and I love to snuggle up close to them. I gave them a big ol' lick on the face, and they laughed and gave me belly rubs.

After that, we went for a walk! Oh boy, I LOVE walks. I got to sniff all the interesting smells and say hello to all the other dogs we met. I'm a bit of a social butterfly (or should I say, social dog?). My tail was wagging the whole time, and I even got to chase a squirrel or two (they're so fast, but it's fun to try to catch them!).

When we got back home, my humans gave me a yummy breakfast of kibble and some scrambled eggs. I gobbled it all up in like two seconds. I'm a bit of a foodie, you know. After breakfast, we played a game of fetch in the backyard. I love chasing after that ball and bringing it back to my humans. They always give me such great praise and belly rubs when I do.

Now I'm just relaxing on my dog b

## Summary

Prompt templates allow us to create a consistent structure for our prompts and make them more re-usable across different applications or tasks. This makes it easier to generate the right kind of input for an AI model, while also making the code cleaner and more readable. 